In [2]:
from keras.layers import Input, Dense, Flatten, Reshape
from sklearn.feature_extraction import image as sk_image
from concurrent.futures import ProcessPoolExecutor
from keras.models import Model
import os
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from sklearn.cluster import AgglomerativeClustering
from scipy.signal import convolve2d
from sklearn.cluster import KMeans
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from pyhdf.SD import SD, SDC
import matplotlib as mpl
#tf.config.threading.set_inter_op_parallelism_threads(1)

from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d
from pyhdf.error import HDF4Error



In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

bands = [1,2]

#bands = [6, 7, 20, 28, 28, 31]


1 Physical GPUs, 1 Logical GPUs


2023-09-15 15:20:27.414516: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21336 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:25:00.0, compute capability: 7.5


In [5]:
#folder = "/uio/hume/student-u37/fslippe/data/training_data/MOD02QKM/normalized_data/"
folder = "/uio/hume/student-u37/fslippe/data/nird_mount/MOD02QKM_202012-202104/normalized_data/"

#folder = "/uio/hume/student-u37/fslippe/data/nird_mount/MOD02QKM_202012-202104/normalized_data/"

all_files = [f for f in os.listdir(folder) if f.endswith('.npz')]

def process_file(file):
    print("file")
    loaded = np.load(folder + file)
    arrays_from_file = [loaded[key][:,:,0] for key in loaded]# if loaded[key].shape[0] >= 256]
    return arrays_from_file

# Use a process pool to process each file in parallel
with ProcessPoolExecutor(max_workers = len(all_files)) as executor:
    X_lists = list(executor.map(process_file, all_files))

# Flatten the result
X = [item for sublist in X_lists for item in sublist]


file


In [15]:
import autoencoder
import importlib
importlib.reload(autoencoder)
from autoencoder import SobelFilterLayer, SimpleAutoencoder
bands = [1, 2]
patch_size = 256

autoencoder = SimpleAutoencoder(len(bands), patch_size, patch_size)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

model = autoencoder.model(X, optimizer=optimizer, threshold=0.09,loss="combined")
folder = "/uio/hume/student-u37/fslippe/data/nird_mount/MOD02QKM_202012-202104/normalized_data/"

all_files = [f for f in os.listdir(folder) if f.endswith('.npz')][:5]

for file in all_files:
    data = process_file(file)
    for i in range(len(data)):
        patches = autoencoder.extract_patches(data[i])
        print(patches.shape)
        model.fit(patches, patches, epochs=10)


Input should already be normalized. Call self.normalize to normalize list of data
(651, 256, 256, 2)
Epoch 1/10


2023-09-15 15:02:36.467207: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202


21/21 [==============================] - 6s 57ms/step - loss: 0.2157
Epoch 2/10
21/21 [==============================] - 1s 42ms/step - loss: 0.0956
Epoch 3/10
21/21 [==============================] - 1s 41ms/step - loss: 0.0067
Epoch 4/10
21/21 [==============================] - 1s 40ms/step - loss: 0.0034
Epoch 5/10
21/21 [==============================] - 1s 40ms/step - loss: 0.0021
Epoch 6/10
21/21 [==============================] - 1s 41ms/step - loss: 0.0018
Epoch 7/10
21/21 [==============================] - 1s 41ms/step - loss: 0.0017
Epoch 8/10
21/21 [==============================] - 1s 41ms/step - loss: 0.0017
Epoch 9/10
21/21 [==============================] - 1s 41ms/step - loss: 0.0016
Epoch 10/10
21/21 [==============================] - 1s 42ms/step - loss: 0.0016
(483, 256, 256, 2)
Epoch 1/10
16/16 [==============================] - 1s 57ms/step - loss: 0.0485
Epoch 2/10
16/16 [==============================] - 1s 40ms/step - loss: 0.0485
Epoch 3/10
16/16 [=============

In [4]:
#from autoencoder import SobelFilterLayer, SimpleAutoencoder
import autoencoder
import importlib
importlib.reload(autoencoder)
from autoencoder import SobelFilterLayer, SimpleAutoencoder

bands = [1, 2]
patch_size = 256
autoencoder = SimpleAutoencoder(1, patch_size, patch_size)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)


autoencoder.fit(X, epochs=200, batch_size=32, optimizer=optimizer, threshold=0.09,loss="combined")
# print(X[0].shape)
# #autoencoder = simple_autoencoder([data_01], patch_size)
# autoencoder = simple_autoencoder(1, (2040, 1354), patch_size)    
# autoencoder.fit(X, epochs=5, batch_size=256)

Input should already be normalized. Call self.normalize to normalize list of data
Extracting patches...
Extracting image 0 of 30
Extracting image 1 of 30
Extracting image 2 of 30
Extracting image 3 of 30
Extracting image 4 of 30
Extracting image 5 of 30
Extracting image 6 of 30
Extracting image 7 of 30
Extracting image 8 of 30
Extracting image 9 of 30
Extracting image 10 of 30
Extracting image 11 of 30
Extracting image 12 of 30
Extracting image 13 of 30
Extracting image 14 of 30
Extracting image 15 of 30
Extracting image 16 of 30
Extracting image 17 of 30
Extracting image 18 of 30
Extracting image 19 of 30
Extracting image 20 of 30
Extracting image 21 of 30
Extracting image 22 of 30
Extracting image 23 of 30
Extracting image 24 of 30
Extracting image 25 of 30
Extracting image 26 of 30
Extracting image 27 of 30
Extracting image 28 of 30
Extracting image 29 of 30
Patches shape:  (3301, 256, 256, 1)
Epoch 1/200


2023-09-15 15:17:08.930181: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202


104/104 [==============================] - 9s 37ms/step - loss: 0.0417
Epoch 2/200
104/104 [==============================] - 4s 36ms/step - loss: 0.0101
Epoch 3/200
104/104 [==============================] - 4s 36ms/step - loss: 0.0079
Epoch 4/200
104/104 [==============================] - 4s 37ms/step - loss: 0.0075
Epoch 5/200
104/104 [==============================] - 4s 37ms/step - loss: 0.0073
Epoch 6/200
104/104 [==============================] - 4s 39ms/step - loss: 0.0072
Epoch 7/200
104/104 [==============================] - 4s 35ms/step - loss: 0.0071
Epoch 8/200
104/104 [==============================] - 4s 36ms/step - loss: 0.0070
Epoch 9/200
104/104 [==============================] - 4s 38ms/step - loss: 0.0070
Epoch 10/200
104/104 [==============================] - 4s 36ms/step - loss: 0.0069
Epoch 11/200
104/104 [==============================] - 4s 37ms/step - loss: 0.0069
Epoch 12/200
104/104 [==============================] - 4s 38ms/step - loss: 0.0068
Epoch 13/200


KeyboardInterrupt: 

In [ ]:
autoencoder.autoencoder.save("/uio/hume/student-u37/fslippe/data/models/test_5day_autoencoder")
autoencoder.encoder.save("/uio/hume/student-u37/fslippe/data/models/test_5day_encoder")


In [ ]:
from autoencoder import SobelFilterLayer, SimpleAutoencoder
patch_size = 256
from tensorflow.keras.models import load_model
autoencoder_predict = SimpleAutoencoder(len(bands), patch_size, patch_size)

encoder = load_model("/uio/hume/student-u37/fslippe/data/models/test_5day_encoder")

In [ ]:
#autoencoder.autoencoder = model

X_test = X

cluster_map = autoencoder_predict.kmeans(X_test, n_clusters=10, encoder=encoder)


In [ ]:
np.max(X_test[2],axis=(0,1))

In [ ]:
print(all_files)

In [ ]:
from matplotlib.colors import Normalize

# Determine global min and max labels
global_min = np.min([np.min(cm) for cm in cluster_map])
global_max = np.max([np.max(cm) for cm in cluster_map])

norm = Normalize(vmin=global_min, vmax=global_max)

for i in range(len(cluster_map)):
    fig, axs = plt.subplots(1, 2, figsize=[10, 8])
    
    cb = axs[0].imshow(cluster_map[i], cmap="tab10", norm=norm)
    plt.colorbar(cb, ax=axs[0])
    
    cb = axs[1].imshow(X_test[i][:, :, 0])
    plt.colorbar(cb, ax=axs[1])

    plt.tight_layout()


In [ ]:
folder = "/uio/hume/student-u37/fslippe/data/cao/"
i=0
print(folder + all_files[i])
hdf = SD(folder + all_files[i], SDC.READ)
print(hdf.attributes())
lon = hdf.select('Longitude')[:]
lat = hdf.select('Latitude')[:]
downsample_factor_x = int(lon.shape[0] / cluster_map[i].shape[0])
downsample_factor_y = int(lon.shape[1] / cluster_map[i].shape[1])
cluster_shape = cluster_map[i].shape
lon_d = lon[::downsample_factor_x, ::downsample_factor_y][:cluster_shape[0], :cluster_shape[1]]
lat_d = lat[::downsample_factor_x, ::downsample_factor_y][:cluster_shape[0], :cluster_shape[1]]
data = hdf.select("EV_250_Aggr1km_RefSB")[:][0]
print(data.shape)
hdf.select("EV_250_Aggr1km_RefSB").attributes()
print(lon.shape)
cluster_map[i].shape
datasets = hdf.datasets()   
for idx, sds in enumerate(datasets.keys()):
    print(idx, sds)




In [ ]:
fix, axs= plt.subplots(1,2)
axs[0].imshow(cluster_map[i])
axs[1].imshow(X[i][:,:,0])


In [ ]:
projection = ccrs.NorthPolarStereo()
fig, axs = plt.subplots(1, 2, figsize=[10,5], subplot_kw={'projection': projection}, dpi=300)
cb = axs[0].pcolormesh(lon_d, lat_d, cluster_map[i], transform=ccrs.PlateCarree(), cmap="jet")  
axs[0].coastlines()
gl = axs[0].gridlines(draw_labels=True)
plt.colorbar(cb)

cb = axs[1].pcolormesh(lon, lat, data, transform=ccrs.PlateCarree(), vmin=0)  
axs[1].coastlines()
gl = axs[1].gridlines(draw_labels=True)
plt.colorbar(cb)
plt.show()

In [ ]:
# patches_predict = autoencoder.extract_patches(dataset_5.reshape((1,dataset_5.shape[0], dataset_5.shape[1], 1)))
# predict = autoencoder.autoencoder.predict(patches_predict)
# predict.shape

# #for i in range(83, 90, 2):
# pictures = 12
# patch_start = 68
# fig, axs = plt.subplots(2, pictures, figsize=[pictures*2,5])
# i=0
# for patch_number in range(patch_start, patch_start+pictures):
#     axs[0, i].imshow(patches_predict[patch_number], cmap="gray")
#     axs[1, i].imshow(predict[patch_number,:,:,0], cmap="gray")
    
#     # Turn off ticks and axis labels for both x and y
#     axs[0, i].set_yticks([])
#     axs[0, i].set_xticks([])
#     axs[0, i].axis('off')
    
#     axs[1, i].set_yticks([])
#     axs[1, i].set_xticks([])
#     axs[1, i].axis('off')
    
#     i += 1
# plt.tight_layout()

patches_predict = autoencoder2.extract_patches(dataset_5.reshape((1,dataset_5.shape[0], dataset_5.shape[1], 1)))
predict = autoencoder2.autoencoder.predict(patches_predict)
print(predict.shape)

#for i in range(83, 90, 2):
pictures = 10
patch_start = 320
fig, axs = plt.subplots(2, pictures, figsize=[pictures*2,5])
i=0
for patch_number in range(patch_start, patch_start+pictures):
    axs[0, i].imshow(patches_predict[patch_number], cmap="gray")
    axs[1, i].imshow(predict[patch_number,:,:,0], cmap="gray")
    
    # Turn off ticks and axis labels for both x and y
    axs[0, i].set_yticks([])
    axs[0, i].set_xticks([])
    axs[0, i].axis('off')
    
    axs[1, i].set_yticks([])
    axs[1, i].set_xticks([])
    axs[1, i].axis('off')
    
    i += 1
plt.tight_layout()

plt.figure()
plt.imshow(dataset_5, cmap='gray')

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

cmp = axes[0].imshow(dataset_2, cmap='gray')
plt.colorbar(cmp, ax=axes[0])
axes[0].set_title('Original Image')
axes[0].axis('off')

cmp = axes[1].imshow(labels_img, cmap='tab10')
plt.colorbar(cmp, ax=axes[1])
axes[1].set_title('Clustered Image')
axes[1].axis('off')

plt.show()

In [ ]:
encoder = Model(flattened_input, encoded)
encoded_imgs = encoder.predict(data)
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Reduce the encoded images to 2D using PCA
pca = PCA(n_components=2)
encoded_imgs_2d = pca.fit_transform(encoded_imgs)

# Now cluster the 2D encoded images
n_clusters = 5  # Define the number of clusters you want
kmeans = KMeans(n_clusters=n_clusters)
cluster_assignments = kmeans.fit_predict(encoded_imgs_2d)

print(cluster_assignments.shape)

# If you want to visualize the clusters using matplotlib:
plt.scatter(encoded_imgs_2d[:, 0], encoded_imgs_2d[:, 1], c=cluster_assignments, cmap='jet')
plt.colorbar()
plt.title('Clusters in 2D')
plt.show()


In [ ]:
decoded_imgs = autoencoder.predict(dataset_2)
fig, axs = plt.subplots(1, 2, figsize=[10,5])

cmp = axs[0].imshow(dataset_2, cmap="gray")
plt.colorbar(cmp)
cmp = axs[1].imshow(decoded_imgs,cmap="gray")
plt.colorbar(cmp)

#decoded_imgs_2d = decoded_imgs.reshape((-1, *input_shape))
# plt.contourf(decoded_imgs_2d)
# plt.show(


In [ ]:
decoded_imgs_reshaped = decoded_imgs.reshape(-1, 1)

n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(decoded_imgs_reshaped)
labels = kmeans.labels_
labels_reshaped = labels.reshape(2040, 1354)

# Assuming `original_img` is your original image
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

cmp = axes[0].imshow(dataset_2, cmap='gray')  # or just cmap depending on the nature of your image
plt.colorbar(cmp)
axes[0].set_title('Original Image')
axes[0].axis('off')

cmp = axes[1].imshow(labels_reshaped, cmap='tab10')  # 'tab10' is a colormap with distinct colors
plt.colorbar(cmp)
axes[1].set_title('Clustered Image')
axes[1].axis('off')

plt.show()


In [ ]:

# Parameters
block_size = (10, 10)  # Change accordingly
n_clusters = 5

# Divide the image into blocks and calculate the mean for each block
blocks = []
for i in range(0, decoded_imgs.shape[0], block_size[0]):
    for j in range(0, decoded_imgs.shape[1], block_size[1]):
        block = decoded_imgs[i:i+block_size[0], j:j+block_size[1]]
        block_mean = np.mean(block)
        blocks.append(block_mean)

blocks = np.array(blocks).reshape(-1, 1)

# Cluster the blocks
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(blocks)
labels = kmeans.labels_

# Construct the clustered image
clustered_img = np.zeros_like(decoded_imgs)
label_idx = 0
for i in range(0, decoded_imgs.shape[0], block_size[0]):
    for j in range(0, decoded_imgs.shape[1], block_size[1]):
        clustered_img[i:i+block_size[0], j:j+block_size[1]] = labels[label_idx]
        label_idx += 1

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Assuming `original_img` is your original image
axes[0].imshow(dataset_2, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(clustered_img, cmap='tab10')  # 'tab10' for distinct colors
axes[1].set_title('Clustered Image')
axes[1].axis('off')

plt.show()